In [2]:

from langchain.chat_models import ChatOpenAI
# 테스트 파일,파워포인트, html, pdfs, images , 등등 로드 할 수 있다.
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
#임베딩 캐시하기
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

cache_embeddings = CacheBackedEmbeddings

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap= 100,
)

loader = UnstructuredFileLoader("./files/chapter_one.pdf")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings,cache_dir)

vectorstore = FAISS.from_documents(docs, cache_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n{context}"),
    ("human", "{question}"),
        
])

chain = {"context" :retriver, "question" : RunnablePassthrough()} | prompt | llm
#Describe Victory Mansions 이건 retriver 에 사용됨 
chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a building where Winston Smith resides. It has glass doors at the entrance and is located in London, the chief city of Airstrip One. The hallway of Victory Mansions has a smell of boiled cabbage and old rag mats. Inside the building, there is a coloured poster depicting an enormous face of a man with a black mustache and ruggedly handsome features. The flat where Winston lives is on the seventh floor, and the lift is seldom working. The building is surrounded by a grimy landscape with rotting nineteenth-century houses, patched windows, and sagging garden walls.')